In [111]:
#initial visualisation with file name
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

# 設定資料夾路徑
#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output'

input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output'


# 確保輸出資料夾存在，如果不存在就創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 設定固定的最小和最大值
fixed_vmin = 10  # 替換為您的固定最小值
fixed_vmax = 18  # 替換為您的固定最大值

#1 10-18
#3 7.5-12

# 列舉資料夾中的所有CSV檔案
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # 構建CSV檔案的完整路徑
        csv_file_path = os.path.join(input_folder, filename)

        # 讀取CSV檔案並跳過第一行
        data = []
        with open(csv_file_path, 'r') as file:
            csv_reader = csv.reader(file)
            
            # 跳過第一行
            next(csv_reader)
            
            for row in csv_reader:
                cleaned_row = []
                for value in row:
                    try:
                        cleaned_value = float(value.strip()) if value.strip() else 0.0
                        cleaned_row.append(cleaned_value)
                    except ValueError:
                        print(f"Warning: Non-numeric value '{value}' found in file {filename}. Skipping this value.")
                        cleaned_row.append(0.0)  # Or you can choose to skip this row entirely
                data.append(cleaned_row)

        # 檢查是否有有效數據
        if not data:
            print(f"No valid data found in {filename}. Skipping this file.")
            continue

        # 將數據轉換為NumPy數組
        matrix = np.array(data)

        # 檢查NumPy數組的形狀
        if matrix.size == 0:
            print(f"Matrix is empty for {filename}. Skipping this file.")
            continue

        # 彩色可視化，使用 viridis 色彩映射
        plt.imshow(matrix, cmap='viridis', vmin=fixed_vmin, vmax=fixed_vmax)

        plt.colorbar()
        plt.title('Matrix Visualisation')

        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]
        plt.annotate(file_datetime_str, (0.5, -0.1), xycoords="axes fraction", ha="center", fontsize=8, color='black')

        # 生成輸出檔案路徑
        output_file_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.jpg')

        # 保存圖像為JPEG檔案
        plt.savefig(output_file_path)

        # 關閉當前圖形以釋放資源，防止內存泄漏
        plt.close()

print('Visualisation completed.')


Visualization completed.


In [11]:
pip install imageio

     -------------------------------------- 313.2/313.2 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [112]:
#Synthesis as a gif animation
import os
import imageio
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# 設定資料夾路徑
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output'

# 確保輸出資料夾存在
if not os.path.exists(output_folder):
    print(f"Output folder '{output_folder}' does not exist.")
    exit()

# 列舉資料夾中的所有 JPEG 檔案
images = []
for filename in os.listdir(output_folder):
    if filename.endswith('.jpg'):
        file_path = os.path.join(output_folder, filename)

        # 讀取圖片，將 NumPy 數組轉換為 Pillow 圖片
        img = Image.fromarray(imageio.imread(file_path))

        # 在圖片上添加文字（檔案名稱）
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()
        
        # 取得檔案名稱的日期和時間，去除 '_temp'
        base_filename = os.path.splitext(filename)[0]
        if base_filename.endswith('_temp'):
            base_filename = base_filename[:-5]
        
        file_datetime = datetime.strptime(base_filename, '%Y_%m_%d_%H_%M_%S_%f')

        # 在圖片上添加文字
        draw.text((10, 10), file_datetime.strftime('%Y-%m-%d %H:%M:%S.f'), (255, 255, 255), font=font)

        images.append(img)

# 設定 GIF 檔案的輸出路徑
gif_output_path = os.path.join(output_folder, 'animation-overall.gif')

# 將多個 JPEG 檔案合併為 GIF 動畫
imageio.mimsave(gif_output_path, images, duration=0.5)

print(f'Animation saved at {gif_output_path}.')



Animation saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\animation-overall.gif.


In [113]:
#Calculate the average of each csv file for build the regression model first without edges


import os
import pandas as pd

def calculate_average_for_file(csv_file):
    print(f"Processing file: {csv_file}")
    
    try:
        # Read the CSV file, excluding the first column
        df = pd.read_csv(csv_file, usecols=lambda col: col != 'ColumnNameToDelete')

        # Exclude the first and 11 rows, and the latest 10 rows
        df = df.iloc[11:-10, :]

        # Exclude the first and latest 10 columns
        df = df.iloc[:, 10:-10]
        
        # Calculate the average
        file_average = df.mean().mean()
        print(f"Average value for {os.path.basename(csv_file)}: {file_average}")

        return {'Filename': os.path.basename(csv_file), 'Average': file_average}
    except Exception as e:
        print(f"Error processing {os.path.basename(csv_file)}: {e}")
        return {'Filename': os.path.basename(csv_file), 'Average': None}

    #directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1'
    directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2'
    #directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3'
    

# Get a list of all CSV files in the specified directory
file_list = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Process each CSV file and accumulate results
average_results = []
for csv_file in file_list:
    result = calculate_average_for_file(csv_file)
    average_results.append(result)

# Create a DataFrame from the accumulated results
result_df = pd.DataFrame(average_results)

# Save the results to a new CSV file

#Output_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output'
Output_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output'
#Output_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output'
result_csv_file = os.path.join(Output_path, 'average_results.csv')
result_df.to_csv(result_csv_file, index=False)
print(f"Combined average results saved to: {result_csv_file}")



Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_42_614_temp.csv
Average value for 2024_06_12_09_57_42_614_temp.csv: 8.427886710239651
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_52_36_973_temp.csv
Average value for 2024_06_12_09_52_36_973_temp.csv: 9.876397966594046
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_52_37_480_temp.csv
Average value for 2024_06_12_09_52_37_480_temp.csv: 10.028380537400144
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_52_37_974_temp.csv
Average value for 2024_06_12_09_52_37_974_temp.csv: 9.932047930283224
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_52_38_490_temp.csv
Average value for 2024_06_12_09_52_38_490_temp.csv: 10.504843863471311
Processing file: Z:\USR\CO2 camera\202

Average value for 2024_06_12_09_53_11_218_temp.csv: 11.794466230936818
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_11_723_temp.csv
Average value for 2024_06_12_09_53_11_723_temp.csv: 12.265250544662306
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_12_231_temp.csv
Average value for 2024_06_12_09_53_12_231_temp.csv: 11.099999999999998
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_12_731_temp.csv
Average value for 2024_06_12_09_53_12_731_temp.csv: 11.588242556281772
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_13_252_temp.csv
Average value for 2024_06_12_09_53_13_252_temp.csv: 11.121968046477852
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_13_749_temp.csv
Average value for 2024_06_12_09_53_

Average value for 2024_06_12_09_53_33_061_temp.csv: 9.832999273783587
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_33_575_temp.csv
Average value for 2024_06_12_09_53_33_575_temp.csv: 10.118177196804641
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_34_072_temp.csv
Average value for 2024_06_12_09_53_34_072_temp.csv: 9.85464778503994
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_34_589_temp.csv
Average value for 2024_06_12_09_53_34_589_temp.csv: 9.727305737109658
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_35_087_temp.csv
Average value for 2024_06_12_09_53_35_087_temp.csv: 9.40100944081336
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_35_600_temp.csv
Average value for 2024_06_12_09_53_35_600

Average value for 2024_06_12_09_53_54_868_temp.csv: 10.87861292665214
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_55_367_temp.csv
Average value for 2024_06_12_09_53_55_367_temp.csv: 10.34280319535221
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_55_880_temp.csv
Average value for 2024_06_12_09_53_55_880_temp.csv: 10.4916630355846
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_56_380_temp.csv
Average value for 2024_06_12_09_53_56_380_temp.csv: 10.27450254175744
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_56_894_temp.csv
Average value for 2024_06_12_09_53_56_894_temp.csv: 10.808402323892519
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_53_57_395_temp.csv
Average value for 2024_06_12_09_53_57_39

Average value for 2024_06_12_09_54_16_710_temp.csv: 10.076797385620914
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_54_17_260_temp.csv
Average value for 2024_06_12_09_54_17_260_temp.csv: 10.2478140885984
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_54_17_767_temp.csv
Average value for 2024_06_12_09_54_17_767_temp.csv: 9.241147421931736
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_54_18_267_temp.csv
Average value for 2024_06_12_09_54_18_267_temp.csv: 8.470987654320986
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_54_18_781_temp.csv
Average value for 2024_06_12_09_54_18_781_temp.csv: 9.513870733478575
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_54_19_288_temp.csv
Average value for 2024_06_12_09_54_19_28

...

Average value for 2024_06_12_09_56_05_985_temp.csv: 10.663856209150321
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_06_493_temp.csv
Average value for 2024_06_12_09_56_06_493_temp.csv: 10.252418300653591
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_06_999_temp.csv
Average value for 2024_06_12_09_56_06_999_temp.csv: 9.685221496005815
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_07_499_temp.csv
Average value for 2024_06_12_09_56_07_499_temp.csv: 10.238366013071893
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_08_014_temp.csv
Average value for 2024_06_12_09_56_08_014_temp.csv: 10.842316630355842
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_08_515_temp.csv
Average value for 2024_06_12_09_56_0

Average value for 2024_06_12_09_56_27_825_temp.csv: 9.180021786492373
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_28_325_temp.csv
Average value for 2024_06_12_09_56_28_325_temp.csv: 9.641764705882348
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_28_839_temp.csv
Average value for 2024_06_12_09_56_28_839_temp.csv: 9.912970225127086
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_29_343_temp.csv
Average value for 2024_06_12_09_56_29_343_temp.csv: 9.63647785039942
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_29_853_temp.csv
Average value for 2024_06_12_09_56_29_853_temp.csv: 9.775257806826433
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_30_353_temp.csv
Average value for 2024_06_12_09_56_30_353

Average value for 2024_06_12_09_56_49_731_temp.csv: 9.632113289760348
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_50_243_temp.csv
Average value for 2024_06_12_09_56_50_243_temp.csv: 9.238663761801014
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_50_759_temp.csv
Average value for 2024_06_12_09_56_50_759_temp.csv: 9.503413217138707
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_51_272_temp.csv
Average value for 2024_06_12_09_56_51_272_temp.csv: 10.127712418300652
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_51_773_temp.csv
Average value for 2024_06_12_09_56_51_773_temp.csv: 10.366688453159039
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_56_52_286_temp.csv
Average value for 2024_06_12_09_56_52_

Average value for 2024_06_12_09_57_11_633_temp.csv: 8.883405954974585
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_12_147_temp.csv
Average value for 2024_06_12_09_57_12_147_temp.csv: 9.29325344952796
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_12_646_temp.csv
Average value for 2024_06_12_09_57_12_646_temp.csv: 8.483057371096585
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_13_181_temp.csv
Average value for 2024_06_12_09_57_13_181_temp.csv: 8.905867828612925
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_13_689_temp.csv
Average value for 2024_06_12_09_57_13_689_temp.csv: 9.161445170660857
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_14_213_temp.csv
Average value for 2024_06_12_09_57_14_213

Average value for 2024_06_12_09_57_33_471_temp.csv: 10.717109658678286
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_33_985_temp.csv
Average value for 2024_06_12_09_57_33_985_temp.csv: 10.906122004357295
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_34_489_temp.csv
Average value for 2024_06_12_09_57_34_489_temp.csv: 10.373042846768335
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_35_001_temp.csv
Average value for 2024_06_12_09_57_35_001_temp.csv: 10.178090050835147
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_35_500_temp.csv
Average value for 2024_06_12_09_57_35_500_temp.csv: 9.011147421931737
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\2024_06_12_09_57_36_013_temp.csv
Average value for 2024_06_12_09_57_3

In [ ]:
# 以下選定chamber區域計算

In [127]:
# Crop raw data to fit chamber zone
## 直的是column
## 橫的是row
### columnXrow=(1+480)X 640

import os
import pandas as pd

def process_csv_file(csv_file, output_directory):
    print(f"Processing file: {csv_file}")

    try:
        # Read the CSV file, excluding the first row
        df = pd.read_csv(csv_file, skiprows=[0])

        # Crop the DataFrame to keep only the desired region
        # up, bottom, left, right, 
        
        #middle zone 
        #cropped_df = df.iloc[181:-200, 440:-100]
        
        #upper zone
        #cropped_df = df.iloc[81:-300, 440:-100]
        
        #lower zone
        #cropped_df = df.iloc[51:-50, 340:-250] for 1
        #cropped_df = df.iloc[51:-50, 400:-190] for 3
        cropped_df = df.iloc[51:-50, 380:-210]

        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        # Save the cropped DataFrame to a new CSV file in the output directory
        output_file = os.path.join(output_directory, os.path.basename(csv_file))
        cropped_df.to_csv(output_file, index=False)
        print(f"Cropped and numbers added to the first column in {os.path.basename(csv_file)}. Saved to: {output_file}")
    except Exception as e:
        print(f"Error processing {os.path.basename(csv_file)}: {e}")


#directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1'
#output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output chamber CSV'        
        
directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2'
output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV'

#directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3'
#output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output chamber CSV'


# Get a list of all CSV files in the specified directory
file_list = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Process each CSV file, crop, and add numbers to the first column
for csv_file in file_list:
    process_csv_file(csv_file, output_directory)

print(f"Cropped and numbers added to the first column in all CSV files. Output saved to: {output_directory}")


Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_08_096_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_08_096_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_08_096_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_08_612_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_08_612_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_08_612_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_09_126_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_09_126_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber

Cropped and numbers added to the first column in 2024_06_12_10_01_20_311_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_20_311_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_20_831_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_20_831_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_20_831_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_21_341_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_21_341_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_21_341_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_01_32_497_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_32_497_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_32_995_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_32_995_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_32_995_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_33_509_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_33_509_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_33_509_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_01_44_664_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_44_664_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_45_163_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_45_163_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_45_163_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_45_677_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_45_677_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_45_677_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_01_56_833_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_56_833_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_57_346_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_57_346_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_57_346_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_57_851_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_57_851_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_01_57_851_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

...

Cropped and numbers added to the first column in 2024_06_12_10_02_21_183_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_21_183_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_21_683_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_21_683_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_21_683_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_22_197_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_22_197_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_22_197_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_02_33_351_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_33_351_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_33_852_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_33_852_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_33_852_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_34_365_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_34_365_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_34_365_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_02_45_521_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_45_521_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_46_021_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_46_021_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_46_021_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_46_533_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_46_533_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_46_533_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_02_57_688_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_57_688_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_58_202_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_58_202_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_58_202_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_58_705_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_58_705_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_02_58_705_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

Cropped and numbers added to the first column in 2024_06_12_10_03_09_857_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_03_09_857_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_03_10_370_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_03_10_370_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_03_10_370_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_03_10_870_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_03_10_870_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV\2024_06_12_10_03_10_870_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\202406

In [128]:
#Chamber visualisation with file name
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# 設定資料夾路徑

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output chamber zone images'

input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber zone images'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output chamber zone images'


# 確保輸出資料夾存在，如果不存在就創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 列舉資料夾中的所有CSV檔案
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # 構建CSV檔案的完整路徑
        csv_file_path = os.path.join(input_folder, filename)

        # 讀取CSV檔案
        data = []
        with open(csv_file_path, 'r') as file:
            csv_reader = csv.reader(file)
            
            # 跳過第一行
            next(csv_reader)
            
            for row in csv_reader:
                # 檢查值是否為非空字符串
                cleaned_row = [float(value) if value.strip() != '' else 0.0 for value in row]
                data.append(cleaned_row)

        # 檢查是否有有效數據
        if not data:
            print(f"No valid data found in {filename}. Skipping this file.")
            continue

        # 將數據轉換為NumPy數組
        matrix = np.array(data)

        # 檢查NumPy數組的形狀
        if matrix.size == 0:
            print(f"Matrix is empty for {filename}. Skipping this file.")
            continue

        # 彩色可視化，使用 viridis 色彩映射
        plt.imshow(matrix, cmap='viridis', vmin=10, vmax=18)
        
        #10-18 for 1st
        #7.5-12 for 3rd

        plt.colorbar()
        plt.title('Matrix Visualisation')

        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]
        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加日期和時間註釋
        plt.annotate(formatted_datetime, (0.5, 0), xycoords="figure fraction", ha="center", fontsize=8, color='black')

        # 生成輸出檔案路徑
        output_file_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.jpg')

        # 保存圖像為JPEG檔案
        plt.savefig(output_file_path)

        # 關閉當前圖形以釋放資源，防止內存泄漏
        plt.close()

print('Visualisation completed.')


Visualization completed.


In [129]:
#Synthesis chamber images as a gif animation
import os
import imageio
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# 設定資料夾路徑

#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output chamber zone images'

output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber zone images'

#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output chamber zone images'


# 確保輸出資料夾存在
if not os.path.exists(output_folder):
    print(f"Output folder '{output_folder}' does not exist.")
    exit()

# 列舉資料夾中的所有 JPEG 檔案
images = []
for filename in os.listdir(output_folder):
    if filename.endswith('.jpg'):
        file_path = os.path.join(output_folder, filename)

        # 讀取圖片，將 NumPy 數組轉換為 Pillow 圖片
        img = Image.fromarray(imageio.imread(file_path))

        # 在圖片上添加文字（檔案名稱）
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()
        
        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]

        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加文字
        draw.text((10, 10), formatted_datetime, (255, 255, 255), font=font)

        images.append(img)

# 設定 GIF 檔案的輸出路徑
gif_output_path = os.path.join(output_folder, 'animation-chamber zone.gif')

# 將多個 JPEG 檔案合併為 GIF 動畫
imageio.mimsave(gif_output_path, images, duration=0.5)

print(f'Animation saved at {gif_output_path}.')


Animation saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber zone images\animation-chamber zone.gif.


In [6]:
#transfer from values to ppm per pixel

import os
import pandas as pd

# path

input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output chamber CSV'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output transformed chamber CSV'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output transformed chamber CSV'


os.makedirs(output_folder, exist_ok=True)


for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_folder, filename)
        
        # load CSV
        data = pd.read_csv(file_path)
        
        transformed_data = (0.6609 * data - 5.9273) * 1000
        transformed_data = transformed_data.clip(lower=500)
        
        # Y= (0.6609X -5.9273) *1000 & Y>=500
        
        # save transformed CSV
        output_file = os.path.join(output_folder, filename)
        transformed_data.to_csv(output_file, index=False)

        print(f":transformed CSV saved at {output_file}")


:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_42_614_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_52_36_973_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_52_37_480_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_52_37_974_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_52_38_490_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_52_38

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_15_273_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_15_775_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_16_287_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_16_788_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_17_301_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_17

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_40_168_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_40_671_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_41_172_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_41_684_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_42_184_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_53_42

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_05_056_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_05_559_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_06_067_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_06_570_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_07_083_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_07

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_30_450_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_30_953_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_31_466_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_31_964_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_32_478_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_54_32

...

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_46_157_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_46_672_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_47_172_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_47_686_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_48_187_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_55_48

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_11_058_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_11_555_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_12_069_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_12_570_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_13_086_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_13

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_36_452_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_36_952_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_37_466_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_37_970_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_38_480_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_56_38

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_01_490_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_01_997_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_02_504_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_03_004_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_03_518_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_04

:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_26_379_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_26_887_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_27_388_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_27_901_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_28_401_temp.csv
:transformed CSV saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV\2024_06_12_09_57_28

In [7]:
#transformed chamber visualisation with file name
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# 設定資料夾路徑

input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber CSV'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber zone images'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output transformed chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output transformed chamber zone images'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output transformed chamber CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output transformed chamber zone images'


# 確保輸出資料夾存在，如果不存在就創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 列舉資料夾中的所有CSV檔案
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # 構建CSV檔案的完整路徑
        csv_file_path = os.path.join(input_folder, filename)

        # 讀取CSV檔案
        data = []
        with open(csv_file_path, 'r') as file:
            csv_reader = csv.reader(file)
            
            # 跳過第一行
            next(csv_reader)
            
            for row in csv_reader:
                # 檢查值是否為非空字符串
                cleaned_row = [float(value) if value.strip() != '' else 0.0 for value in row]
                data.append(cleaned_row)

        # 檢查是否有有效數據
        if not data:
            print(f"No valid data found in {filename}. Skipping this file.")
            continue

        # 將數據轉換為NumPy數組
        matrix = np.array(data)

        # 檢查NumPy數組的形狀
        if matrix.size == 0:
            print(f"Matrix is empty for {filename}. Skipping this file.")
            continue

        # 彩色可視化，使用 viridis 色彩映射
        plt.imshow(matrix, cmap='viridis', vmin=500, vmax=5000)

        plt.colorbar()
        plt.title('Matrix Visualisation (CO2 ppm)')

        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]
        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加日期和時間註釋
        plt.annotate(formatted_datetime, (0.5, 0), xycoords="figure fraction", ha="center", fontsize=8, color='black')

        # 生成輸出檔案路徑
        output_file_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.jpg')

        # 保存圖像為JPEG檔案
        plt.savefig(output_file_path)

        # 關閉當前圖形以釋放資源，防止內存泄漏
        plt.close()

print('Visualisation completed.')


Visualisation completed.


In [8]:
#Synthesis chamber images as a gif animation
import os
import imageio
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# 設定資料夾路徑

output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber zone images'

#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output transformed chamber zone images'

#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output transformed chamber zone images'


# 確保輸出資料夾存在
if not os.path.exists(output_folder):
    print(f"Output folder '{output_folder}' does not exist.")
    exit()

# 列舉資料夾中的所有 JPEG 檔案
images = []
for filename in os.listdir(output_folder):
    if filename.endswith('.jpg'):
        file_path = os.path.join(output_folder, filename)

        # 讀取圖片，將 NumPy 數組轉換為 Pillow 圖片
        img = Image.fromarray(imageio.imread(file_path))

        # 在圖片上添加文字（檔案名稱）
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()
        
        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]

        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加文字
        draw.text((10, 10), formatted_datetime, (255, 255, 255), font=font)

        images.append(img)

# 設定 GIF 檔案的輸出路徑
gif_output_path = os.path.join(output_folder, 'animation-transformed chamber zone.gif')

# 將多個 JPEG 檔案合併為 GIF 動畫
imageio.mimsave(gif_output_path, images, duration=0.5)

print(f'Animation saved at {gif_output_path}.')


Animation saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output transformed chamber zone images\animation-transformed chamber zone.gif.


In [121]:
# 以下選定特定區域計算

In [122]:
# Crop raw data to fit specified zone
## 直的是column
## 橫的是row
### columnXrow=(1+480)X 640

import os
import pandas as pd

def process_csv_file(csv_file, output_directory):
    print(f"Processing file: {csv_file}")

    try:
        # Read the CSV file, excluding the first row
        df = pd.read_csv(csv_file, skiprows=[0])

        # Crop the DataFrame to keep only the desired region
        # up, bottom, left, right, 
        
        #middle zone 
        #cropped_df = df.iloc[181:-200, 440:-100]
        
        #upper zone
        #cropped_df = df.iloc[81:-300, 440:-100]
        
        #lower zone
        #cropped_df = df.iloc[381:-50, 340:-250] for 1
        #cropped_df = df.iloc[281:-150, 400:-190] for 3
        cropped_df = df.iloc[381:-50, 380:-210]

        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        # Save the cropped DataFrame to a new CSV file in the output directory
        output_file = os.path.join(output_directory, os.path.basename(csv_file))
        cropped_df.to_csv(output_file, index=False)
        print(f"Cropped and numbers added to the first column in {os.path.basename(csv_file)}. Saved to: {output_file}")
    except Exception as e:
        print(f"Error processing {os.path.basename(csv_file)}: {e}")

#directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1'
#output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output specified CSV'        
        
directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2'
output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV'

#directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3'
#output_directory = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output specified CSV'


# Get a list of all CSV files in the specified directory
file_list = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Process each CSV file, crop, and add numbers to the first column
for csv_file in file_list:
    process_csv_file(csv_file, output_directory)

print(f"Cropped and numbers added to the first column in all CSV files. Output saved to: {output_directory}")


Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_08_096_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_08_096_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_08_096_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_08_612_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_08_612_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_08_612_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_09_126_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_09_126_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output spe

Cropped and numbers added to the first column in 2024_06_12_10_01_20_311_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_20_311_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_20_831_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_20_831_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_20_831_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_21_341_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_21_341_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_21_341_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_01_32_497_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_32_497_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_32_995_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_32_995_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_32_995_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_33_509_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_33_509_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_33_509_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_01_44_664_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_44_664_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_45_163_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_45_163_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_45_163_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_45_677_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_45_677_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_45_677_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_01_56_833_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_56_833_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_57_346_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_57_346_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_57_346_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_01_57_851_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_01_57_851_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_57_851_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

...

Cropped and numbers added to the first column in 2024_06_12_10_02_21_183_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_21_183_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_21_683_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_21_683_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_21_683_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_22_197_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_22_197_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_22_197_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_02_33_351_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_33_351_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_33_852_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_33_852_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_33_852_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_34_365_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_34_365_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_34_365_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_02_45_521_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_45_521_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_46_021_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_46_021_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_46_021_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_46_533_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_46_533_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_46_533_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_02_57_688_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_57_688_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_58_202_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_58_202_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_58_202_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_02_58_705_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_02_58_705_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_58_705_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

Cropped and numbers added to the first column in 2024_06_12_10_03_09_857_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_09_857_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_03_10_370_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_03_10_370_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_10_370_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\2024_06_12_10_03_10_870_temp.csv
Cropped and numbers added to the first column in 2024_06_12_10_03_10_870_temp.csv. Saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_10_870_temp.csv
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\

In [123]:
#Calculate the average of each csv file for build the regression model in specified zone
import os
import pandas as pd

def calculate_statistics_for_file(csv_file):
    print(f"Processing file: {csv_file}")
    
    try:
        # Read the CSV file
        df = pd.read_csv(csv_file)

        # Calculate the average, maximum, and minimum
        file_average = df.mean().mean()
        file_maximum = df.max().max()
        file_minimum = df.min().min()

        print(f"Statistics for {os.path.basename(csv_file)}: "
              f"Average={file_average}, Maximum={file_maximum}, Minimum={file_minimum}")

        return {'Filename': os.path.basename(csv_file), 
                'Average': file_average, 
                'Maximum': file_maximum, 
                'Minimum': file_minimum}
    except Exception as e:
        print(f"Error processing {os.path.basename(csv_file)}: {e}")
        return {'Filename': os.path.basename(csv_file), 
                'Average': None, 
                'Maximum': None, 
                'Minimum': None}

#input_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output specified CSV'
#output_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output'
    
input_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV'
output_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output'

#input_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output specified CSV'
#output_directory_path = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output'


# Get a list of all CSV files in the specified input directory
file_list = [os.path.join(input_directory_path, file) for file in os.listdir(input_directory_path) if file.endswith('.csv')]

# Process each CSV file and accumulate results
statistics_results = []
for csv_file in file_list:
    result = calculate_statistics_for_file(csv_file)
    statistics_results.append(result)

# Create a DataFrame from the accumulated results
result_df = pd.DataFrame(statistics_results)

# Save the results to a new CSV file
result_csv_file = os.path.join(output_directory_path, 'specified_statistics_results.csv')
result_df.to_csv(result_csv_file, index=False)
print(f"Combined statistics results saved to: {result_csv_file}")



Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_08_096_temp.csv
Statistics for 2024_06_12_10_01_08_096_temp.csv: Average=11.052287581699346, Maximum=20.1, Minimum=3.2
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_08_612_temp.csv
Statistics for 2024_06_12_10_01_08_612_temp.csv: Average=11.31168300653595, Maximum=20.7, Minimum=2.2
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_09_126_temp.csv
Statistics for 2024_06_12_10_01_09_126_temp.csv: Average=10.50249183006536, Maximum=18.9, Minimum=2.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_09_627_temp.csv
Statistics for 2024_06_12_10_01_09_627_temp.csv: Average=11.909722222222223, Maximum=20.3, Mi

Statistics for 2024_06_12_10_01_25_399_temp.csv: Average=12.355841503267976, Maximum=21.2, Minimum=4.2
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_25_902_temp.csv
Statistics for 2024_06_12_10_01_25_902_temp.csv: Average=12.025204248366014, Maximum=22.0, Minimum=4.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_26_416_temp.csv
Statistics for 2024_06_12_10_01_26_416_temp.csv: Average=12.450204248366012, Maximum=20.5, Minimum=4.1
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_26_911_temp.csv
Statistics for 2024_06_12_10_01_26_911_temp.csv: Average=28.33799019607843, Maximum=43.6, Minimum=19.7
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_27_4

Statistics for 2024_06_12_10_01_44_151_temp.csv: Average=10.629084967320258, Maximum=20.9, Minimum=1.7
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_44_664_temp.csv
Statistics for 2024_06_12_10_01_44_664_temp.csv: Average=10.706331699346407, Maximum=22.0, Minimum=2.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_45_163_temp.csv
Statistics for 2024_06_12_10_01_45_163_temp.csv: Average=9.720383986928104, Maximum=18.7, Minimum=1.0
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_45_677_temp.csv
Statistics for 2024_06_12_10_01_45_677_temp.csv: Average=9.885089869281044, Maximum=19.1, Minimum=0.4
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_01_46_197

Statistics for 2024_06_12_10_02_00_889_temp.csv: Average=10.823447712418304, Maximum=19.6, Minimum=3.1
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_01_404_temp.csv
Statistics for 2024_06_12_10_02_01_404_temp.csv: Average=10.751511437908498, Maximum=18.7, Minimum=2.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_01_905_temp.csv
Statistics for 2024_06_12_10_02_01_905_temp.csv: Average=10.713562091503272, Maximum=20.7, Minimum=2.6
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_02_416_temp.csv
Statistics for 2024_06_12_10_02_02_416_temp.csv: Average=10.849714052287581, Maximum=19.7, Minimum=1.3
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_02_9

Statistics for 2024_06_12_10_02_17_629_temp.csv: Average=11.381658496732022, Maximum=19.9, Minimum=2.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_18_150_temp.csv
Statistics for 2024_06_12_10_02_18_150_temp.csv: Average=10.869362745098037, Maximum=19.7, Minimum=3.1
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_18_654_temp.csv
Statistics for 2024_06_12_10_02_18_654_temp.csv: Average=11.507434640522872, Maximum=20.0, Minimum=2.6
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_19_156_temp.csv
Statistics for 2024_06_12_10_02_19_156_temp.csv: Average=10.580882352941176, Maximum=18.2, Minimum=1.1
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_19_6

Statistics for 2024_06_12_10_02_41_466_temp.csv: Average=13.337908496732027, Maximum=22.8, Minimum=3.8
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_41_963_temp.csv
Statistics for 2024_06_12_10_02_41_963_temp.csv: Average=12.443014705882353, Maximum=21.9, Minimum=3.9
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_42_477_temp.csv
Statistics for 2024_06_12_10_02_42_477_temp.csv: Average=13.065400326797384, Maximum=23.6, Minimum=5.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_42_977_temp.csv
Statistics for 2024_06_12_10_02_42_977_temp.csv: Average=13.304779411764708, Maximum=23.2, Minimum=4.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_43_4

Statistics for 2024_06_12_10_02_58_705_temp.csv: Average=11.278758169934646, Maximum=19.2, Minimum=2.6
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_59_216_temp.csv
Statistics for 2024_06_12_10_02_59_216_temp.csv: Average=12.077573529411769, Maximum=21.2, Minimum=2.2
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_02_59_722_temp.csv
Statistics for 2024_06_12_10_02_59_722_temp.csv: Average=10.946486928104576, Maximum=19.9, Minimum=1.7
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_00_230_temp.csv
Statistics for 2024_06_12_10_03_00_230_temp.csv: Average=11.737990196078437, Maximum=20.5, Minimum=3.1
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_00_7

Statistics for 2024_06_12_10_03_17_469_temp.csv: Average=12.155473856209147, Maximum=22.8, Minimum=3.8
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_17_968_temp.csv
Statistics for 2024_06_12_10_03_17_968_temp.csv: Average=11.828349673202613, Maximum=23.0, Minimum=2.8
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_18_482_temp.csv
Statistics for 2024_06_12_10_03_18_482_temp.csv: Average=11.82712418300654, Maximum=20.7, Minimum=3.5
Processing file: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV\2024_06_12_10_03_18_982_temp.csv
Statistics for 2024_06_12_10_03_18_982_temp.csv: Average=11.639828431372552, Maximum=22.4, Minimum=2.8
Combined statistics results saved to: Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\specified_statistics_r

In [124]:
#Specified visualisation with file name
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# 設定資料夾路徑

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output specified CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output specified zone images'


input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified CSV'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified zone images'

#input_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output specified CSV'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output specified zone images'


# 確保輸出資料夾存在，如果不存在就創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 列舉資料夾中的所有CSV檔案
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # 構建CSV檔案的完整路徑
        csv_file_path = os.path.join(input_folder, filename)

        # 讀取CSV檔案
        data = []
        with open(csv_file_path, 'r') as file:
            csv_reader = csv.reader(file)
            
            # 跳過第一行
            next(csv_reader)
            
            for row in csv_reader:
                # 檢查值是否為非空字符串
                cleaned_row = [float(value) if value.strip() != '' else 0.0 for value in row]
                data.append(cleaned_row)

        # 檢查是否有有效數據
        if not data:
            print(f"No valid data found in {filename}. Skipping this file.")
            continue

        # 將數據轉換為NumPy數組
        matrix = np.array(data)

        # 檢查NumPy數組的形狀
        if matrix.size == 0:
            print(f"Matrix is empty for {filename}. Skipping this file.")
            continue

        # 彩色可視化，使用 viridis 色彩映射
        plt.imshow(matrix, cmap='viridis', vmin=10, vmax=18)
        #10-18 for 1
        #7.5-12 for 3

        plt.colorbar()
        plt.title('Matrix Visualisation')

        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]
        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加日期和時間註釋
        plt.annotate(formatted_datetime, (0.5, 0), xycoords="figure fraction", ha="center", fontsize=8, color='black')

        # 生成輸出檔案路徑
        output_file_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.jpg')

        # 保存圖像為JPEG檔案
        plt.savefig(output_file_path)

        # 關閉當前圖形以釋放資源，防止內存泄漏
        plt.close()

print('Visualisation completed.')


Visualization completed.


In [125]:
#Synthesis specified images as a gif animation
import os
import imageio
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# 設定資料夾路徑

#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-1\output\Output specified zone images'
output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified zone images'
#output_folder = r'Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-3\output\Output specified zone images'


# 確保輸出資料夾存在
if not os.path.exists(output_folder):
    print(f"Output folder '{output_folder}' does not exist.")
    exit()

# 列舉資料夾中的所有 JPEG 檔案
images = []
for filename in os.listdir(output_folder):
    if filename.endswith('.jpg'):
        file_path = os.path.join(output_folder, filename)

        # 讀取圖片，將 NumPy 數組轉換為 Pillow 圖片
        img = Image.fromarray(imageio.imread(file_path))

        # 在圖片上添加文字（檔案名稱）
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()
        
        # 取得檔案名稱的日期和時間
        file_datetime_str = os.path.splitext(filename)[0]

        # 移除潛在的後綴
        if '_temp' in file_datetime_str:
            file_datetime_str = file_datetime_str.replace('_temp', '')

        try:
            file_datetime = datetime.strptime(file_datetime_str, '%Y_%m_%d_%H_%M_%S_%f')
            formatted_datetime = file_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            formatted_datetime = file_datetime_str  # 如果格式錯誤，直接顯示檔案名稱

        # 在圖片上添加文字
        draw.text((10, 10), formatted_datetime, (255, 255, 255), font=font)

        images.append(img)

# 設定 GIF 檔案的輸出路徑
gif_output_path = os.path.join(output_folder, 'animation-specified zone.gif')

# 將多個 JPEG 檔案合併為 GIF 動畫
imageio.mimsave(gif_output_path, images, duration=0.5)

print(f'Animation saved at {gif_output_path}.')


Animation saved at Z:\USR\CO2 camera\20240612\CO2 camera test 20240612\20240612\20240612-2\output\Output specified zone images\animation-specified zone.gif.
